In [ ]:
# This notebook is used for testing learning in single units using the draculab module.

# In particular, the Oja learning rule is tested by presenting a set of patterns. The learning rule
# is expected to extract the leading eigenvector of the input correlation matrix.
# In the last cell some of the connection methods are tested a bit.

# By Sergio Verduzco Flores        June 2017

In [1]:
# This file is supposed to be in .../draculab/tests/ , so cd before importing:
%cd ..

from draculab import *
import numpy as np
import time
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

/home/z/projects/draculab


In [2]:
def conn_mat(net):
    # Get the connection matrix of a network in a Numpy array
    conns = np.zeros((net.n_units,net.n_units))
    for syn_list in net.syns:
        for syn in syn_list:
            conns[syn.postID,syn.preID] = syn.w
    return conns

def plot_stuff(data, fig):
    fig.clf()
    plt.close()
    fig = plt.figure(figsize=(10,10))
    for i in range(9):
        ax = fig.add_subplot(3,3,i+1)
        ax.clear()
        plt.ylim(-0.01,0.5)
        ax.plot(data[0], data[1][i], 'r', figure=fig)
    fig.canvas.draw()
    plt.show()
    

In [3]:
################# TEST 1 ####################
######### 1) Create a network
net_params = {'min_delay' : 0.05, 'min_buff_size' : 5 } # parameter dictionary for the network
n1 = network(net_params)

######### 2) Put some units in the network
# default parameters for the units
pars = { 'coordinates' : np.array([0., 0., 0.]), 'function' : lambda x : None,
         'delay' : 1., 'init_val' : 0.5, 
         'tau_fast' : 1., 'tau_slow' : 15.,
         'slope' : 1., 'thresh' : 0.0, 'tau' : 0.02,
         'type' : unit_types.source } 
inputs = n1.create(9,pars) # creating nine input sources
pars['type'] = unit_types.linear
unit = n1.create(1,pars) # creating one linear unit

######### 3) Connect the units in the network
conn_spec = {'rule' : 'all_to_all', 'delay' : 1.,
             'allow_autapses' : False} # connection specification dictionary
syn_pars = {'init_w' : {'distribution':'uniform', 'low':0.1, 'high':0.5},
            'omega':1., 'lrate' : 0.02, 
            'type' : synapse_types.oja} # synapse parameters dictionary
n1.connect(inputs, unit, conn_spec, syn_pars)

w_sum = sum([syn.w for syn in n1.syns[unit[0]][0:9]])
sq_w_sum = sum([syn.w*syn.w for syn in n1.syns[unit[0]][0:9]])

######### 4) Running and visualizing 

####### SETTING THE INPUT FUNCTIONS
### You are going to present 4 input patterns that randomly switch over time.
### Imagine the 9 inputs arranged in a grid, like a tic-tac-toe board, numbered
### from left to right and from top to bottom:
### 1 2 3
### 4 5 6
### 7 8 9
### You'll have input patterns
### 0 X 0   0 0 0   X 0 X   0 X 0
### 0 X 0   X X X   0 0 0   X 0 X
### 0 X 0   0 0 0   X 0 X   0 X 0
### The input is always a normalized linear combination of one or two of these patterns.
### Pattern pat1 is presented alone for t_pat time units, and then there is a transition period
### during which pat1 becomes pat2 by presenting at time t an input 
### c*(t_pat+t_trans - t)*pat1 + c*(t - tpat)*pat2
### where c = 1/t_trans, and t_trans is the duration of the transition period. 
### At time t_pat+t_trans, pat2 is presented by itself for t_pat time units.
### 
# here are the patterns as arrays
patterns = [np.zeros(9) for i in range(4)]
patterns[0] = np.array([0., 1., 0., 0., 1., 0., 0., 1., 0.])/3.
patterns[1] = np.array([0., 0., 0., 1., 1., 1., 0., 0., 0.])/3.
patterns[2] = np.array([1., 0., 1., 0., 0., 0., 1., 0., 1.])/4.
patterns[3] = np.array([0., 1., 0., 1., 0., 1., 0., 1., 0.])/4.

n_pres = 120  # number of times some pattern will be presented
t_pat = 10. # as above
t_trans = 4.
c = 1/t_trans # auxiliary variable
cur_pat = np.random.randint(4)  # pattern currently presented
next_pat = np.random.randint(4) # next pattern to be presented
last_t = 0.
weights = np.array(n1.units[9].get_weights(n1.sim_time)) # initial weights of the linear unit
start_time = time.time()

def make_fun1(idx):
    # This creates a constant function with value: patterns[cur_pat][idx]
    # thus avoiding a scoping problem that is sometimes hard to see:
    # https://eev.ee/blog/2011/04/24/gotcha-python-scoping-closures/
    fun = lambda t : patterns[cur_pat][idx]
    return fun

def make_fun2(idx, last_t):
    # Creates a function for the pattern transition
    fun = lambda t : c * ( (t_trans - (t-last_t))*patterns[cur_pat][idx] +
                           (t-last_t)*patterns[next_pat][idx] )
    return fun

fig = plt.figure(figsize=(10,10))
plt.ion()

"""
for pres in range(n_pres):
    # For each cycle you'll set the input functions and simulate, once with a single pattern,
    # once with a mix of patterns, as described above
    # first, we present a single pattern
    for u in range(9):
        n1.units[inputs[u]].set_function( make_fun1(u) )
    sim_dat = n1.flat_run(t_pat)  # simulating
    last_t = n1.sim_time # simulation time after last pattern presentation
    # now one pattern turns into the next
    for u in range(9):
        n1.units[inputs[u]].set_function(make_fun2(u, last_t))
    sim_dat = n1.flat_run(t_trans) # simulating
    # choose the pattern you'll present next
    cur_pat = next_pat
    next_pat = np.random.randint(4)
   
    new_weights = np.array(n1.units[9].get_weights(n1.sim_time))
    weight_diff = new_weights - weights
    weights = new_weights

    if pres%10 == 0:
        print('Presentation ' + str(pres) + ' weight Diff. Sq. norm: ' + str(sum(weight_diff*weight_diff)))

print('Execution time: %s seconds' % (time.time() - start_time))  # around 7.2 seconds
"""

def run(cur_pat, next_pat, last_t, weights, start_time):
    for pres in range(n_pres):
        # For each cycle you'll set the input functions and simulate, once with a single pattern,
        # once with a mix of patterns, as described above
        # first, we present a single pattern
        for u in range(9):
            n1.units[inputs[u]].set_function( make_fun1(u) )
        sim_dat = n1.flat_run(t_pat)  # simulating
        last_t = n1.sim_time # simulation time after last pattern presentation
        # now one pattern turns into the next
        for u in range(9):
            n1.units[inputs[u]].set_function(make_fun2(u, last_t))
        sim_dat = n1.flat_run(t_trans) # simulating
        # choose the pattern you'll present next
        cur_pat = next_pat
        next_pat = np.random.randint(4)
   
        new_weights = np.array(n1.units[9].get_weights(n1.sim_time))
        weight_diff = new_weights - weights
        weights = new_weights

        if pres%10 == 0:
            print('Presentation ' + str(pres) + ' weight Diff. Sq. norm: ' + str(sum(weight_diff*weight_diff)))

    print('Execution time: %s seconds' % (time.time() - start_time))  # around 7.2 seconds

import cProfile
import pstats
cProfile.run('run(cur_pat, next_pat, last_t, weights, start_time)', 'restats')
prof = pstats.Stats('restats')
prof.sort_stats('cumulative').print_stats(60)
    
w_sum2 = sum([syn.w for syn in n1.syns[unit[0]][0:9]])
sq_w_sum2 = sum([syn.w*syn.w for syn in n1.syns[unit[0]][0:9]])
print("w_sum before: %f, w_sum after: %f" % (w_sum, w_sum2))
print("sq_w_sum before: %f, sq_w_sum after: %f" % (sq_w_sum, sq_w_sum2))

0.25
2
Presentation 0 weight Diff. Sq. norm: 0.0011694922396305325
Presentation 10 weight Diff. Sq. norm: 0.00045847182907652646
Presentation 20 weight Diff. Sq. norm: 0.0002212573091945371
Presentation 30 weight Diff. Sq. norm: 8.781789007585733e-05
Presentation 40 weight Diff. Sq. norm: 3.221921431420214e-05
Presentation 50 weight Diff. Sq. norm: 1.1491837134409748e-05
Presentation 60 weight Diff. Sq. norm: 4.061075394330457e-06
Presentation 70 weight Diff. Sq. norm: 1.4315338375670814e-06
Presentation 80 weight Diff. Sq. norm: 5.045041658260332e-07
Presentation 90 weight Diff. Sq. norm: 1.7791369934146644e-07
Presentation 100 weight Diff. Sq. norm: 6.279419269057723e-08
Presentation 110 weight Diff. Sq. norm: 2.218284116291966e-08
Execution time: 15.299052953720093 seconds
Thu May  9 10:26:21 2019    restats

         5118035 function calls in 15.297 seconds

   Ordered by: cumulative time
   List reduced from 76 to 60 due to restriction <60>

   ncalls  tottime  percall  cumtime  p

<Figure size 720x720 with 0 Axes>

In [ ]:
#### Comparing the weight vector with the leading eigenvector of the correlation matrix
#######################################################################################

weights = np.array(n1.units[9].get_weights(n1.sim_time))

# Obtaining eigenvectors of the correlation matrix
#pat_mat = np.matrix(patterns)
#corr = (pat_mat.T)*pat_mat # input correlation matrix
pat_arr = np.array(patterns)
corr = np.matmul(pat_arr.transpose(), pat_arr)
eigs = np.linalg.eig(corr) # extracting eigenvalues and eigenvectors
evals = eigs[0] # eigenvalues
evecs = [eigs[1][:,i] for i in range(9)] # eigenvectors

# obtaining the leading eigenvector
max_index, max_value = max(enumerate(evals), key=lambda p:p[1])
print('Max eigenvalue: ' + str(max_value) + ', index: ' + str(max_index))
max_evector = evecs[max_index]
    
# plotting all eigenvectors in 3x3 format
fig3 = plt.figure(figsize=(10,10))
ev_grid = ImageGrid(fig3, 111, nrows_ncols=(3,3), axes_pad=0.05)
for idx,vec in enumerate(evecs):
    vec.shape = 3,3
    ev_grid[idx].imshow(vec)

# plotting leading eigenvector VS weight vector in 3x3 format
fig4 = plt.figure(figsize=(12,5))
sp_ev_grid = ImageGrid(fig4, 111, nrows_ncols=(1,2), axes_pad=0.2)
for idx,vec in enumerate([-max_evector, weights]):
    vec.shape = 3,3
    sp_ev_grid[idx].imshow(vec)
    print(["%.2f" % float(v[0]) for v in vec.reshape(9,1)])


In [4]:
# I wrote this class to test how the eigenvector closest to the weight vector
# could change depending on the 'tau_slow' time constant when using
# covariance synapses
class eigenvalue_test():
    """ Eigenvalue extraction depending on time constants. """

    def __init__(self):
        self.setUpClass()
        
    def setUpClass(self):
        ####### SETTING THE INPUT FUNCTIONS
        ### You are going to present 4 input patterns that randomly switch over time.
        ### Imagine the 9 inputs arranged in a grid, like a tic-tac-toe board, numbered
        ### from left to right and from top to bottom:
        ### 1 2 3
        ### 4 5 6
        ### 7 8 9
        ### You'll have input patterns
        ### 0 X 0   0 0 0   X 0 X   0 X 0
        ### 0 X 0   X X X   0 0 0   X 0 X
        ### 0 X 0   0 0 0   X 0 X   0 X 0
        ### The input is always a normalized linear combination of one or two of these 
        ### patterns. Pattern pat1 is presented alone for t_pat time units, and then
        ### there is a transition period during which pat1 becomes pat2 by presenting
        ### at time t an input 
        ### c*(t_pat+t_trans - t)*pat1 + c*(t - tpat)*pat2
        ### where c = 1/t_trans, and t_trans is the duration of the transition period. 
        ### At time t_pat+t_trans, pat2 is presented by itself for t_pat time units.
        ### 
        # here are the patterns as arrays
        self.patterns = [np.zeros(9) for i in range(4)]
        self.patterns[0] = np.array([0., 1., 0., 0., 1., 0., 0., 1., 0.])/3.
        self.patterns[1] = np.array([0., 0., 0., 1., 1., 1., 0., 0., 0.])/3.
        self.patterns[2] = np.array([1., 0., 1., 0., 0., 0., 1., 0., 1.])/4.
        self.patterns[3] = np.array([0., 1., 0., 1., 0., 1., 0., 1., 0.])/4.
        ####### THE LEADING EIGENVECTOR
        # The code that obtains this is in test3,4,5.ipynb
        self.max_evector = [0.0000, 0.4316, 0.0000, 0.4316, 0.5048,
                            0.4316, 0.0000, 0.4316, 0.0000]

    def make_fun1(self, idx, cur_pat):
        """ This creates a constant function with value: patterns[cur_pat][idx]
            thus avoiding a scoping problem that is sometimes hard to see:
            https://eev.ee/blog/2011/04/24/gotcha-python-scoping-closures/
        """
        fun = lambda t : self.patterns[cur_pat][idx]
        return fun

    def make_fun2(self, idx, last_t, cur_pat, next_pat, t_trans):
        """ Creates a function for the pattern transition. """
        fun = lambda t : self.c * ( (t_trans - (t-last_t))*self.patterns[cur_pat][idx] +
                            (t-last_t)*self.patterns[next_pat][idx] )
        return fun

    def create_network(self, syn_type=synapse_types.antihebb, tau_slow=15.):
        """ Creates a network for a test. """
        ######### 1) Create a network
        net_params = {'min_delay' : 0.05, 'min_buff_size' : 5 }
        n1 = network(net_params)
        ######### 2) Put some units in the network
        # default parameters for the units
        pars = { 'function' : lambda x : None,
                'delay' : 1., 'init_val' : 0.5,
                'tau_fast' : 1., 'tau_slow' : tau_slow,
                'slope' : 1., 'thresh' : 0.0, 'tau' : 0.02,
                'type' : unit_types.source }
        self.inputs = n1.create(9,pars) # creating nine input sources
        pars['type'] = unit_types.linear
        self.unit = n1.create(1,pars) # creating one linear unit
        ######### 3) Connect the units in the network
        conn_spec = {'rule' : 'all_to_all', 'delay' : 1.,
                    'allow_autapses' : False}
        syn_pars = {'init_w' : {'distribution':'uniform', 'low':0.1, 'high':0.5},
                    'omega' : 1., 'lrate' : 0.02,
                    'type' : syn_type}
        n1.connect(self.inputs, self.unit, conn_spec, syn_pars)
        ######## 4) Return
        self.net = n1

    def run_net(self, n_pres=120, t_pat=10., t_trans=3.):
        """ Simulate 'n_pres' pattern presentations.

        Each pattern is presented for t_pat time unts, and transitions
        between patterns last t_trans time units.
        """
        self.c = 1/t_trans # auxiliary variable
        cur_pat = np.random.randint(4)  # pattern currently presented
        next_pat = np.random.randint(4) # next pattern to be presented
        last_t = 0.

        for pres in range(n_pres):
        # For each cycle you'll set the input functions and simulate, once with
        # a single pattern, # once with a mix of patterns, as described above
            # first, we present a single pattern
            for u in range(9):
                self.net.units[self.inputs[u]].set_function( self.make_fun1(u, cur_pat) )
            sim_dat = self.net.flat_run(t_pat)  # simulating
            last_t = self.net.sim_time # simulation time after last pattern presentation
            # now one pattern turns into the next
            for u in range(9):
                self.net.units[self.inputs[u]].set_function(self.make_fun2(u, last_t, cur_pat, next_pat, t_trans))
            sim_dat = self.net.flat_run(t_trans) # simulating
            # choose the pattern you'll present next
            cur_pat = next_pat
            next_pat = np.random.randint(4)
            
    def get_evecs(self):
        # obtains evals, evecs, max_evector. Plots evecs in 9x1 format
        weights = np.array(self.net.units[9].get_weights(self.net.sim_time))
        # Obtaining eigenvectors of the correlation matrix
        pat_arr = np.array(self.patterns)
        corr = np.matmul(pat_arr.transpose(), pat_arr)
        eigs = np.linalg.eig(corr) # extracting eigenvalues and eigenvectors
        self.evals = eigs[0] # eigenvalues
        self.evecs = [eigs[1][:,i] for i in range(9)] # eigenvectors
        # plotting all eigenvectors in 9x1 format
        #fig_evecs = plt.figure(figsize=(12,12))
        #ax_evecs = plt.gca()
        #ax_evecs.imshow(self.evecs)
        #plt.colorbar(ax_evecs) ???
        # obtaining the leading eigenvector
        max_index, max_value = max(enumerate(self.evals), key=lambda p:p[1])
        #print('Max eigenvalue: ' + str(max_value) + ', index: ' + str(max_index))
        self.max_evector = self.evecs[max_index]
        self.max_evector.shape = 1,9
            
    def plot_evecs(self):
        # plots weights in 9x1, all evecs in 3x3, and leading vs weights in 3x3.
        # plotting the weights in 9x1 format
        weights = np.array(self.net.units[9].get_weights(self.net.sim_time))
        weights.shape = 9,1
        fig_max_ev = plt.figure(figsize=(11.5,2))
        ax_max_ev = plt.gca()
        ax_max_ev.imshow(weights.transpose())
        # plotting all eigenvectors in 3x3 format
        fig3 = plt.figure(figsize=(10,10))
        ev_grid = ImageGrid(fig3, 111, nrows_ncols=(3,3), axes_pad=0.05)
        for idx,vec in enumerate(self.evecs):
            vec.shape = 3,3
            ev_grid[idx].imshow(vec)
        # plotting leading eigenvector VS weight vector in 3x3 format
        fig4 = plt.figure(figsize=(12,5))
        sp_ev_grid = ImageGrid(fig4, 111, nrows_ncols=(1,2), axes_pad=0.2)
        for idx,vec in enumerate([-self.max_evector, weights]):
            vec.shape = 3,3
            sp_ev_grid[idx].imshow(vec)
            print(["%.2f" % float(v[0]) for v in vec.reshape(9,1)])

    def plot_top(self):
        # plot weights vector, evector with max eig, and evector closest to weights
        C = conn_mat(self.net)
        C[-2,:-1] = self.max_evector.flatten() # maximum eigenvector second to last
        C[-3,:-1] = self.evecs[self.most_idx].flatten() # closest eigenvector at top
        labels=["closest:", "largest:", "weights:"]
        for i,row in enumerate(C[7:]):
            print(labels[i], end=' ')
            print(["{0:0.1f}".format(i) for i in row])
        fig5 = plt.figure(figsize=(14,4))
        conn_grid = ImageGrid(fig5, 111, nrows_ncols=(1,1), axes_pad=0.2)
        conn_grid[0].imshow(C[7:])
        
    def get_angles(self):
        # obtain the cosine of the angle between the weights vector and the eigenvectors
        weights = np.array(self.net.units[9].get_weights(self.net.sim_time))
        weights.shape = 9,1
        #print(weights)
        self.max_evector.shape=9,1
        self.max_evector = -self.max_evector
        #print(self.max_evector)
        cos = sum(self.max_evector*weights) / (np.linalg.norm(self.max_evector)*np.linalg.norm(weights))
        #print("angle with leading eigenvector: %f" % (cos))
        # obtain cosine of angle with other eigenvectors
        weights = weights.transpose()
        self.cozz = np.zeros(9)
        for i, rowevecs in enumerate(self.evecs):
            vec = np.transpose(rowevecs.flatten())
            icozz = np.sum(vec*weights) / (np.linalg.norm(vec)*np.linalg.norm(weights))
            self.cozz[i] = icozz
            #print("index = %d:  %f" % (i, icozz))
        self.most_idx = np.argmax(np.abs(self.cozz))
        print("index of most aligned evec: %d " % (self.most_idx))
        #print("Indexes arranged in order of eigenvalue size: ")
        #print(np.argsort(self.evals))
               
    def test_range(self):
        # obtain index of closest eigenvalue for a range of parameter values
        tau_slows = [3., 6., 9.] #, 12., 15., 18., 21., 24., 27., 100.]
        closest = np.zeros_like(tau_slows)
        for i, t_s in enumerate(tau_slows):
            self.create_network(syn_type=synapse_types.cov, tau_slow=t_s)
            self.run_net(n_pres=120, t_pat=10., t_trans=5.)
            self.get_evecs()
            self.get_angles()
            closest[i] = self.most_idx
        print(closest)           

In [5]:
evt = eigenvalue_test()
import cProfile
import pstats
cProfile.run('evt.test_range()', 'restats')
prof = pstats.Stats('restats')
prof.sort_stats('cumulative').print_stats(60)
#evt.test_range()

index of most aligned evec: 1 
index of most aligned evec: 2 
index of most aligned evec: 2 
[1. 2. 2.]
Thu May  9 10:12:08 2019    restats

         17742331 function calls (17742328 primitive calls) in 51.957 seconds

   Ordered by: cumulative time
   List reduced from 188 to 60 due to restriction <60>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   51.957   51.957 {built-in method builtins.exec}
        1    0.000    0.000   51.957   51.957 <ipython-input-4-9e39d9df6e5b>:185(test_range)
        3    0.021    0.007   51.945   17.315 <ipython-input-4-9e39d9df6e5b>:80(run_net)
      720    1.472    0.002   51.854    0.072 /home/z/projects/draculab/network.py:1011(flat_run)
   108000    8.768    0.000   45.928    0.000 /home/z/projects/draculab/network.py:975(flat_update)
  1080000    1.341    0.000   12.105    0.000 /home/z/projects/draculab/units/units.py:575(pre_syn_update)
   972000    5.341    0.000   10.246    0.000 /home/z/pr

In [ ]:
Thu May  9 09:46:25 2019    restats

         36756817 function calls (35568583 primitive calls) in 138.789 seconds

   Ordered by: cumulative time
   List reduced from 436 to 60 due to restriction <60>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     27/1    0.001    0.000  138.789  138.789 {built-in method builtins.exec}
        1    0.000    0.000  138.789  138.789 <string>:1(<module>)
        1    0.000    0.000  138.789  138.789 <ipython-input-3-9e39d9df6e5b>:185(test_range)
        3    0.021    0.007  138.748   46.249 <ipython-input-3-9e39d9df6e5b>:80(run_net)
      720    1.693    0.002  138.654    0.193 /home/z/projects/draculab/network.py:1011(flat_run)
   108000   11.182    0.000  132.189    0.001 /home/z/projects/draculab/network.py:975(flat_update)
  1080000    1.936    0.000   92.764    0.000 /home/z/projects/draculab/units/units.py:575(pre_syn_update)
  1080000    9.913    0.000   84.570    0.000 /home/z/projects/draculab/units/units.py:694(upd_lpf_fast)
2376000/1188000   37.654    0.000   52.966    0.000 /home/z/.local/lib/python3.5/site-packages/numpy/core/numeric.py:1318(roll)
  1944000   29.043    0.000   29.043    0.000 <ipython-input-3-9e39d9df6e5b>:51(<lambda>)
   972000    6.635    0.000   11.861    0.000 /home/z/projects/draculab/network.py:1000(<listcomp>)
  1188000    4.863    0.000    7.367    0.000 /home/z/.local/lib/python3.5/site-packages/numpy/core/numeric.py:1468(normalize_axis_tuple)
   108000    1.086    0.000    6.257    0.000 /home/z/projects/draculab/units/units.py:734(upd_lpf_slow)
   108000    4.741    0.000    5.793    0.000 /home/z/projects/draculab/units/units.py:1019(upd_flat_inp_sum)
   972000    3.592    0.000    5.248    0.000 /home/z/projects/draculab/synapses/synapses.py:220(update)
  1080000    3.266    0.000    4.744    0.000 /home/z/projects/draculab/network.py:950(get_act)
  3456222    4.293    0.000    4.293    0.000 {built-in method numpy.core.multiarray.array}
  2376036    2.874    0.000    3.476    0.000 /home/z/.local/lib/python3.5/site-packages/numpy/core/numeric.py:504(asanyarray)
  2376000    1.574    0.000    2.272    0.000 /home/z/.local/lib/python3.5/site-packages/numpy/core/numeric.py:1515(<genexpr>)
  1296000    2.132    0.000    2.132    0.000 {cython_utils.cython_get_act3}
   108000    1.476    0.000    2.110    0.000 /home/z/projects/draculab/units/units.py:1066(flat_euler_update)
  3890835    1.974    0.000    1.974    0.000 <ipython-input-3-9e39d9df6e5b>:46(<lambda>)
  1188000    1.496    0.000    1.496    0.000 {method 'reshape' of 'numpy.ndarray' objects}
  1944000    1.094    0.000    1.094    0.000 /home/z/projects/draculab/units/units.py:710(get_lpf_fast)
   216000    0.417    0.000    1.071    0.000 /home/z/projects/draculab/units/units.py:521(get_act)
  1188001    1.039    0.000    1.039    0.000 {built-in method numpy.core.multiarray.empty_like}
  1188060    0.867    0.000    0.867    0.000 {method 'ravel' of 'numpy.ndarray' objects}
  1188000    0.782    0.000    0.782    0.000 /home/z/.local/lib/python3.5/site-packages/numpy/core/numeric.py:1387(<dictcomp>)
  1188000    0.698    0.000    0.698    0.000 {built-in method numpy.core.multiarray.normalize_axis_index}
   540000    0.634    0.000    0.634    0.000 /home/z/projects/draculab/units/units.py:1356(dt_fun)
   972000    0.562    0.000    0.562    0.000 /home/z/projects/draculab/units/units.py:747(get_lpf_slow)
   108003    0.301    0.000    0.301    0.000 {built-in method numpy.core.multiarray.matmul}
   108000    0.293    0.000    0.293    0.000 /home/z/projects/draculab/units/units.py:1033(<listcomp>)
  1188040    0.287    0.000    0.287    0.000 {method 'items' of 'dict' objects}
  1188018    0.232    0.000    0.232    0.000 {built-in method _operator.index}
     6480    0.062    0.000    0.062    0.000 /home/z/projects/draculab/units/units.py:1160(set_function)
        3    0.000    0.000    0.038    0.013 <ipython-input-3-9e39d9df6e5b>:55(create_network)
        6    0.000    0.000    0.030    0.005 /home/z/projects/draculab/network.py:62(create)
        6    0.000    0.000    0.029    0.005 /home/z/projects/draculab/network.py:116(create_units)
     32/2    0.000    0.000    0.029    0.015 <frozen importlib._bootstrap>:966(_find_and_load)
     32/2    0.000    0.000    0.029    0.015 <frozen importlib._bootstrap>:939(_find_and_load_unlocked)
     31/2    0.000    0.000    0.028    0.014 <frozen importlib._bootstrap>:659(_load_unlocked)
     21/2    0.000    0.000    0.028    0.014 <frozen importlib._bootstrap_external>:659(exec_module)
        6    0.000    0.000    0.028    0.005 /home/z/projects/draculab/draculab.py:65(get_class)
     50/2    0.000    0.000    0.027    0.013 <frozen importlib._bootstrap>:214(_call_with_frames_removed)
        1    0.000    0.000    0.026    0.026 /home/z/projects/draculab/units/units.py:4(<module>)
        1    0.000    0.000    0.024    0.024 /home/z/.local/lib/python3.5/site-packages/scipy/interpolate/__init__.py:172(<module>)
      720    0.005    0.000    0.022    0.000 /home/z/projects/draculab/network.py:1022(<listcomp>)
        1    0.000    0.000    0.020    0.020 /home/z/.local/lib/python3.5/site-packages/scipy/interpolate/interpolate.py:2(<module>)
     7927    0.018    0.000    0.018    0.000 {built-in method numpy.core.multiarray.zeros}
    31/19    0.000    0.000    0.015    0.001 <frozen importlib._bootstrap>:570(module_from_spec)
     10/5    0.000    0.000    0.014    0.003 <frozen importlib._bootstrap_external>:903(create_module)
     10/5    0.005    0.000    0.014    0.003 {built-in method _imp.create_dynamic}
      6/4    0.000    0.000    0.012    0.003 <frozen importlib._bootstrap>:1043(__import__)
     12/8    0.000    0.000    0.012    0.002 <frozen importlib._bootstrap>:972(_gcd_import)
        1    0.000    0.000    0.011    0.011 /home/z/.local/lib/python3.5/site-packages/scipy/spatial/__init__.py:90(<module>)
        3    0.000    0.000    0.008    0.003 /home/z/projects/draculab/network.py:234(connect)
   114/99    0.000    0.000    0.007    0.000 <frozen importlib._bootstrap>:996(_handle_fromlist)
       21    0.000    0.000    0.007    0.000 <frozen importlib._bootstrap_external>:729(get_code)
      366    0.007    0.000    0.007    0.000 {method 'randint' of 'mtrand.RandomState' objects}


In [ ]:
evt = eigenvalue_test()
evt.create_network()
evt.run_net()
evt.get_evecs()
evt.plot_evecs()
evt.get_angles()
evt.plot_top()

In [8]:
a = np.array([1,3,4,5])
a[:-1] = a[1:]
a

array([3, 4, 5, 5])

In [ ]:
# testing conn_spec rules and syn_par specifications
net_params = {'min_delay' : 0.1, 'min_buff_size' : 4 } # parameter dictionary for the network
n2 = network(net_params)

######### 2) Put some units in the network
# default parameters for the units
pars = { 'coordinates' : np.array([0., 0., 0.]), 'function' : lambda x:None,
         'delay' : 1., 'init_val' : 0.5, 'tau_fast' : 1.,
         'slope' : 1., 'thresh' : 0.0, 'tau' : 0.02,
         'type' : unit_types.source } 
inputs = n2.create(4,pars) # creating input sources
pars['type'] = unit_types.sigmoidal
sigs = n2.create(10,pars) # creating sig units

######### 3) Connect the units in the network
conn_spec = {'rule' : 'one_to_one', 'indegree' : 4, 'outdegree' : 5, 'delay' : 1.,
             'allow_autapses' : True, 'allow_multapses' : True } # connection specification dictionary
syn_pars = {'init_w' : 1, 'lrate' : 0.01, 
            'type' : synapse_types.oja} # synapse parameters dictionary
n2.connect(inputs, sigs[2:6], conn_spec, syn_pars)
#print(conn_mat(n2))
syn_pars['init_w'] = {"distribution" : 'uniform', 'low' : 0.1, 'high' : 0.7}
conn_spec['rule'] = 'fixed_indegree'
n2.connect(sigs, sigs, conn_spec, syn_pars)
C = conn_mat(n2)
for row in C:
    print(["{0:0.1f}".format(i) for i in row])

fig5 = plt.figure(figsize=(12,12))
conn_grid = ImageGrid(fig5, 111, nrows_ncols=(1,1), axes_pad=0.2)
conn_grid[0].imshow(C)


In [ ]:
for syn in n2.syns[sigs[9]]:
    print(" (%d,%d)" % (syn.preID, syn.postID))